In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from sensor import BatterySavingSensor

plt.rcParams["figure.dpi"] = 100

# Load data

In [ ]:
# Read data from parquet file
data = pd.read_parquet("raw_data_all.parquet")

# For simplicity, select sensor 3
data = data[data["sensor"] == "node_03"]

# Downsample to 1-minute medians
data = data.resample("1min").median()

# For simplicity, select only week 7
data = data[data.index.week.isin([7, 8])]

data.head()

# Simulate behaviour

In [ ]:
# Create a sensor and simulate behaviour
sensor = BatterySavingSensor(max_period="3 hour", deadband=2)
sensor.simulate(data["Leq"])

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 9))
resampled = data["Leq"].resample("1h")
ax[0].fill_between(
    resampled.indices.keys(), resampled.min(), resampled.max(),
    alpha=0.3,
    label="Real world"
)
ax[0].plot(sensor.measured, "g.", markersize=3, label="Measurement")
ax[0].plot(sensor.transmitted, "r.", markersize=3, label="Transmission")
ax[0].legend(loc="upper right")

ax[1].fill_between(
    resampled.indices.keys(), resampled.min(), resampled.max(),
    alpha=0.3,
    label="Real world"
)
ax[1].step(sensor.transmitted, "r", where="post", label="Recorded signal")
ax[1].legend(loc="upper right")

In [ ]:
# Resample real and transmitted signal to same frequency
real = data["Leq"].resample("2s").median()
meas = sensor.transmitted.resample("2s").median().fillna(method='ffill')

# RMSE
((real - meas)**2).mean()**.5

In [ ]:
# Average period between measurements
(data.index[-1] - data.index[0]) / len(sensor.measured) 

In [ ]:
# Average period between tranmissions
(data.index[-1] - data.index[0]) / len(sensor.transmitted) 